In [1]:
import torch
import simtext
from torch import nn
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import gc
import os
import pickle
from sklearn.preprocessing import Normalizer
import datetime
import pyecharts
import warnings
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# import mplcursors
from matplotlib.animation import HTMLWriter
from pyecharts import options as opts
from pyecharts.globals import SymbolType
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Line, Scatter, EffectScatter
from pyecharts.charts import *
from pyecharts.options import *
#from util import *
#from util.check_utils import dir_check
from pyecharts.globals import WarningType
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
import re
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso,ElasticNet
#import yfinance as yf
#import pyfolio as pf
from matplotlib.ticker import FuncFormatter
#from sklearn.linear_model import LinearRegression
import pymysql, time
import cx_Oracle
import pymysql
import jsdata as js
import jieba



time_1 = datetime.datetime.now()
warnings.filterwarnings('ignore')

plt.rcParams["font.family"]='SimHei'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['lines.markersize'] = 15
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['figure.figsize'] = (18,8)   
plt.rcParams['figure.dpi'] = 300.0  # 分辨率
plt.rcParams['savefig.dpi'] = 300.0  # 图片像素

harvest_red = '#a50000'
harvest_gray = '#b5b6b6'

warnings.filterwarnings('ignore')
######################################################################
os.chdir('./data')
######################################################################
if 'instantclient_19_5' not in os.environ['PATH']:
    os.environ['PATH'] = 'C:\\Users\\XCZGY\\Anaconda3\\instantclient_19_5;' + os.environ['PATH']
os.environ["NLS_LANG"] = "SIMPLIFIED CHINESE_CHINA.UTF8"

Oracle_USER = "jrkj"
Oracle_PWD = 'bpkdJ4_atXFJ7'
Oracle_HOST = '10.0.185.137'
Oracle_PORT = '1521'
Oracle_SERV = 'winddb'

conn_addr = Oracle_USER + '/' + Oracle_PWD + '@' + Oracle_HOST + ':' + Oracle_PORT + '/' + Oracle_SERV
wind_conn = cx_Oracle.connect(conn_addr)

题目数据导入

In [2]:
#事先将港股，A股打上标签
data_set = pd.read_excel('数据.xlsx',sheet_name ='已有分类A股标的')
predict_set = pd.read_excel('数据.xlsx',sheet_name ='待分类港股标的')
predict_set['A/H'] = 1
data_set['A/H'] = 0

In [3]:
#经过测试 单纯使用GICS分类进行处理的效果就不错
#而题目中也说明可以加入自己的数据。因此我们引入 wind分类
#A股
sql = 'select S_INFO_WINDCODE,Wind_ind_code from wind.AShareIndustriesClass where CUR_SIGN = 1'
A_wind_indus = pd.read_sql(sql,wind_conn)
A_wind_indus['wind一级'] = A_wind_indus['WIND_IND_CODE'].apply(lambda x:x[:4]).astype(int)
A_wind_indus['wind二级'] = A_wind_indus['WIND_IND_CODE'].apply(lambda x:x[:6]).astype(int)
A_wind_indus['wind三级'] = A_wind_indus['WIND_IND_CODE'].apply(lambda x:x[:8]).astype(int)
A_wind_indus = A_wind_indus.rename(columns = {'WIND_IND_CODE':'wind四级','S_INFO_WINDCODE':'股票代码'})
A_wind_indus['wind四级'] = A_wind_indus['wind四级'].astype(int)
data_set = pd.merge(data_set,A_wind_indus,on = '股票代码',how = 'left')
#港股
sql = 'select S_INFO_WINDCODE,Wind_ind_code from wind.HKStockWindIndustriesMembers where CUR_SIGN = 1'
HK_wind_indus = pd.read_sql(sql,wind_conn)
HK_wind_indus['wind一级'] = HK_wind_indus['WIND_IND_CODE'].apply(lambda x:x[:4]).astype(int)
HK_wind_indus['wind二级'] = HK_wind_indus['WIND_IND_CODE'].apply(lambda x:x[:6]).astype(int)
HK_wind_indus['wind三级'] = HK_wind_indus['WIND_IND_CODE'].apply(lambda x:x[:8]).astype(int)
HK_wind_indus = HK_wind_indus.rename(columns = {'WIND_IND_CODE':'wind四级','S_INFO_WINDCODE':'股票代码'})
HK_wind_indus['wind四级'] = HK_wind_indus['wind四级'].astype(int)
predict_set = pd.merge(predict_set,HK_wind_indus,on = '股票代码',how = 'left')

In [4]:
#由于存在两地上市 因此我们要检查一下是否给定的数据中存在这种股票
sql = 'select S_INFO_WINDCODE as A股证券代码, S_INFO_WINDCOD2 as 港股证券代码  from  wind.SHSZRelatedsecurities'
cross_board_list = pd.read_sql(sql,wind_conn)
cross_board_list = pd.merge(cross_board_list,
                            data_set[['股票代码','一级行业','二级行业']].rename(columns = {'股票代码':'A股证券代码','一级行业':'拟一级行业','二级行业':'拟二级行业'}),
                            on = 'A股证券代码',
                            how = 'left')
predict_set_have_label = pd.merge(predict_set,
                       cross_board_list.rename(columns = {'港股证券代码':'股票代码'}),
                       on = '股票代码',
                       how = 'left')
predict_set_have_label = predict_set_have_label[~predict_set_have_label['拟二级行业'].isna()].drop_duplicates()
columns_name = ['股票代码', '股票简称', '公司简介', 'GICS一级行业', 'GICS二级行业','A/H',
       'GICS三级行业', 'wind四级', 'wind一级', 'wind二级', 'wind三级', 'A股证券代码', '拟一级行业',
       '拟二级行业']
predict_set_have_label = predict_set_have_label[columns_name]
predict_set_have_label=predict_set_have_label.rename(columns = {'拟一级行业':'一级行业','拟二级行业':'二级行业'})[data_set.columns]

In [5]:
#事实上的确存在。这部分同样可以作为训练集加入训练。或者作为测试集。能够有效避免过拟合
predict_set = predict_set[~predict_set['股票代码'].isin(predict_set_have_label['股票代码'])]
data_set = data_set.append(predict_set_have_label)
# predict_set_have_label.head(5)

预测

In [6]:
data_set_1 = data_set[['股票代码', '股票简称', '公司简介', '一级行业',
                       '二级行业','A/H', 'GICS一级行业', 'GICS二级行业','GICS三级行业'
                      , 'wind一级', 'wind二级', 'wind三级','wind四级']]
# 一级指标明细
label_list_1 = list(data_set_1['一级行业'].unique())
# 一级指标编号 用于模型训练
label_list_1 = pd.DataFrame(label_list_1)
label_1 = label_list_1.reset_index().rename(columns = {0:'一级行业','index':'label_1'})
data_set_1 = pd.merge(data_set_1,label_1,on = '一级行业',how = 'left')
# 二级指标明细
label_list_2 = list(data_set_1['二级行业'].unique())
# 二级指标编号 用于模型训练
label_list_2 = pd.DataFrame(label_list_2)
label_2 = label_list_2.reset_index().rename(columns = {0:'二级行业','index':'label_2'})
data_set_1 = pd.merge(data_set_1,label_2,on = '二级行业',how = 'left')
label_total = pd.merge(pd.merge(data_set_1[['label_1','label_2']].drop_duplicates(),label_1,on = 'label_1',how = 'left'),label_2,on = 'label_2',how = 'left')

In [7]:
#公司简介中有为nan的存在，先清除nan 再对简介进行分词

print(data_set_1[data_set_1['公司简介'].isna()])
data_set_1 =data_set_1[~data_set_1['公司简介'].isna()]
data_set_1 = data_set_1.dropna()

          股票代码   股票简称 公司简介  一级行业  二级行业  A/H GICS一级行业  GICS二级行业    GICS三级行业  \
140  000587.SZ  *ST金洲  NaN  商贸零售  珠宝首饰    0     可选消费  耐用消费品与服装  纺织品、服装与奢侈品   

     wind一级  wind二级    wind三级      wind四级  label_1  label_2  
140    6225  622520  62252030  6225203010       11       13  


In [8]:
# 进行分词
def trans_intr(x):
    x = jieba.cut(x)
    x = list(x)
    return x
data_set_1['公司简介(分词)'] = data_set_1['公司简介'].transform(trans_intr)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.760 seconds.
Prefix dict has been built successfully.


In [9]:
#去停用词
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='UTF-8').readlines()]
    return stopwords
stoplist1 = stopwordslist('hit_stopwords.txt')
stoplist2 = stopwordslist('cn_stopwords.txt')
stoplist3 = stopwordslist('baidu_stopwords.txt')
#经过对分词结果的评析 我们列出了另外一个词频表
stoplist4 = ['公司', '企业', '产品', '生产', '研发', '技术', '中国', '服务', '主要', '国家', '业务', '行业',
       '领域', '发展', '销售', '国内', '拥有', '提供', '客户', '制造', '高新技术', '中心', '年', '开发',
       '认证', '品牌', '设计', '市场', '管理', '系统', '设备',  '包括', '创新', '专业', '全国',
       '从事', '智能', '一家', '国际', '应用',  '经营', '成为', '产业', '工业', '解决方案',
       '核心', '获得', '全球', '重点', '中', '质量', '新', '工程', '领先', '项目', '形成', '具有',
       '单位', '自主', '先进', '目前', '科技', '综合', '集团', '建设', '能力', '平台', '优势', '运营',
       '多年', '致力于', '研究', '管理体系', '先后', '装备', '主营业务', '安全', '实现', '投资',
       '建立', '多项', '战略', '多个', '供应商', '体系', '合作', '系列', '大型', '认定', '国家级',
       '专注', '基地', '一体', '城市','上市','更名','持续', '不断', '有限公司', '江苏省','坚持', '上市公司', '优秀','集',
                       '完全',  '高端', '示范','方面', '荣获', '提升', '涵盖','国内外', '打造','配套', '布局', '浙江省',
       '规模', '广东省', '模式', '最大', '积累', '积极', '评为', '控制','知名', '竞争', '整合','知名', '名牌', '标准', '驰名商标', '现代'
             ,'品质', '产业化', '地区','广泛','竞争', '基础', '互联', '广泛', '知名', '覆盖','竞争力', '营销', '著名','世界', 
             '地区', '荣誉', '为主', '竞争', '商标','世界', '完善', '标准', '一体化', '工作', '资源', '水平']
stoplist5 = ['上海', '依托', '区域','高效', '称号','控股','第一', 
 '实验', '协会','成立','科学', '美国', '计划', '始终', '影响', '快速', '理念',
  '重要','工艺', '荣誉称号','集成', '经验', '取得','日本', 'ISO9001', '相关','长期', '4.0']
num = pd.DataFrame(data_set_1['公司简介(分词)'].explode().unique())
stoplist6 = num[num[0].str.contains('^[0-9]*$')][0].to_list()
stoplist7=num[num[0].str.contains('^[A-Z]*$')][0].to_list()
stoplist8=num[num[0].str.contains('^[a-z]*$')][0].to_list()


stop_word_list_general = stoplist1+stoplist2+stoplist3+stoplist4+stoplist5+stoplist6+stoplist7+stoplist8
stop_word_list_general = list(set(stop_word_list_general))
def delete_stop_word_general(list1):
    list_done  = []
    for i in list1:
        if i not in stop_word_list_general:
            list_done.append(i)
    return list_done
data_set_1['公司简介(分词) 去 停用'] = data_set_1['公司简介(分词)'].transform(delete_stop_word_general)

In [10]:
keywords_selection = pd.DataFrame(data_set_1.groupby(['二级行业'])['公司简介(分词) 去 停用'])
keywords_selection[2] = keywords_selection[1].apply(lambda x:','.join(x.explode().to_list()))
keywords_selection[3] = keywords_selection[1].apply(lambda x:','.join(list(set(x.explode().to_list()))))
keywords_selection['keywords'] = pd.Series()
for i in range(len(keywords_selection)):
    input_1 = keywords_selection[1].iloc[i]
    one_sector = pd.DataFrame(input_1)
    one_sector['merge'] = one_sector['公司简介(分词) 去 停用'].apply(lambda x:','.join(list(set(x))))
    tf = pd.DataFrame(index = one_sector['公司简介(分词) 去 停用'].explode().unique()).reset_index()
    tf = tf[tf['index'].apply(lambda x:len(x)) >=2]
    tf['tf'] = tf['index'].apply(lambda x:one_sector['merge'].str.contains(x).sum()/len(one_sector))
    tf['idf'] = tf['index'].apply(lambda x:len(keywords_selection)/(keywords_selection[3].str.contains(x).sum()+1))
    tf['tf-idf'] = tf['tf']/tf['idf']
    keywords_selection['keywords'].iloc[i] = tf.sort_values('tf-idf')['index'].tail(20).to_list()


In [11]:
#关键词整合
keywords_list = keywords_selection.groupby(0)['keywords'].sum().explode().unique()
def trans2vec(x):
    return pd.merge(pd.DataFrame(pd.DataFrame(index = keywords_list)),
    pd.DataFrame(x).reset_index().groupby(0).count(),
    left_index = True,
    right_index = True,
    how = 'left').fillna(0)['index'].to_list()
trans_result = pd.DataFrame(data_set_1['公司简介(分词) 去 停用'].transform(trans2vec).to_list())
trans_result.columns = keywords_list



In [12]:
data_set_1 = pd.merge(data_set_1,trans_result,how = 'left',left_index = True,right_index = True)

In [13]:
for i in ['GICS一级行业', 'GICS二级行业', 'GICS三级行业']:
    data_set_1=pd.merge(data_set_1,
         pd.DataFrame(list(data_set_1[i].unique())).reset_index().rename(columns = {0:i,'index':i+'label'}),
         on = i,
         how = 'left')

In [14]:
keywords_list = list(keywords_list)

In [15]:
#一级的预测效果90% 二级如果用和一级一样的指标只有60% 
#因此我们先预测一级 把预测的模型也加入第二部的预测。因为一级只能对应一些行业 能够提高预测准确率
train_data_set  = data_set_1[data_set_1['A/H'] == 0]
test_data_set = data_set_1[data_set_1['A/H'] == 1]
# test_data_set =  test_data_set[~test_data_set['股票代码'].isin(test_data_set_sample['股票代码'])]
X_train = train_data_set[['wind一级', 'wind二级', 'wind三级','wind四级','GICS一级行业label', 'GICS二级行业label','GICS三级行业label']+list(keywords_list)]
y_train = train_data_set['label_1'].copy()
X_test = test_data_set[['wind一级', 'wind二级', 'wind三级','wind四级','GICS一级行业label', 'GICS二级行业label','GICS三级行业label']+list(keywords_list)]
y_test = test_data_set['label_1'].copy()
clf1 = xgb.XGBClassifier()
clf1.fit(X_train,y_train)
test_predict = clf1.predict(X_test)
confusion_matrix_result = metrics.confusion_matrix(test_predict,y_test)
print(np.trace(confusion_matrix_result)/len(test_predict))

0.9148936170212766


In [16]:
input_for_first = list(X_train.columns)
X_train['pre'] = clf1.predict(X_train)
X_test['pre'] = clf1.predict(X_test)
y_train = train_data_set['label_2'].copy()
y_test = test_data_set['label_2'].copy()
#将一级指标对应的二级信息加入数据集中
secodary = pd.get_dummies(label_total['label_2'])
secodary.index = label_total['label_1']
secodary = secodary.reset_index().groupby('label_1').sum()
secodary = secodary.reset_index().rename(columns = {'label_1':'pre'})

In [17]:
X_train = pd.merge(X_train,secodary,on = 'pre',how = 'left')
X_test = pd.merge(X_test,secodary,on = 'pre',how = 'left')
input_for_secode = list(X_train.columns)

In [18]:
clf2 = xgb.XGBClassifier()
clf2.fit(X_train,y_train)
test_predict = clf2.predict(X_test)
confusion_matrix_result = metrics.confusion_matrix(test_predict,y_test)
print(np.trace(confusion_matrix_result)/len(test_predict))

0.6808510638297872


计算最终结果

In [19]:
x = predict_set.sort_values('股票代码')
X_pre = x[['股票代码','GICS一级行业', 'GICS二级行业','GICS三级行业','wind一级', 'wind二级', 'wind三级','wind四级']]
x['公司简介(分词)'] = x['公司简介'].transform(trans_intr)
x['公司简介(分词) 去 停用'] = x['公司简介(分词)'].transform(delete_stop_word_general)
trans_result_for_test = pd.DataFrame(x['公司简介(分词) 去 停用'].transform(trans2vec).to_list())
trans_result_for_test.columns = keywords_list
X_pre = pd.merge(X_pre,trans_result_for_test,how = 'left',left_index = True,right_index = True)
for i in ['GICS一级行业', 'GICS二级行业', 'GICS三级行业']:
    X_pre=pd.merge(X_pre,
         pd.DataFrame(list(data_set_1[i].unique())).reset_index().rename(columns = {0:i,'index':i+'label'}),
         on = i,
         how = 'left')


In [20]:
#一级预测
output_result_1 = clf1.predict(X_pre[input_for_first])

In [41]:
X_pre['pre'] = output_result_1 
X_pre = pd.merge(X_pre,secodary,on = 'pre',how = 'left')
output_result_2 = clf1.predict(X_pre[input_for_secode])

KeyError: '[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144] not in index'

In [22]:
#结果输出
output_result_for_2 = pd.merge(pd.DataFrame(output_result_2,X_pre['股票代码']).reset_index().rename(columns = {0:'label_2'}),
label_total,
on = 'label_2',
how = 'left')[['股票代码','二级行业']]

output_result_for_1 = pd.merge(pd.DataFrame(output_result_1,X_pre['股票代码']).reset_index().rename(columns = {0:'label_1'}),
label_total,
on = 'label_1',
how = 'left')[['股票代码','一级行业']]
output = pd.merge(output_result_for_2,output_result_for_1,on = '股票代码',how = 'left').drop_duplicates()
output = output[['股票代码','一级行业','二级行业']].append(predict_set_have_label[['股票代码','一级行业','二级行业']])
predict_set = pd.read_excel('数据.xlsx',sheet_name ='待分类港股标的')
output = pd.merge(predict_set.drop(['一级行业','二级行业'],axis = 1),output,on = '股票代码',how = 'left')[predict_set.columns]

In [23]:
output.to_excel('预测结果.xlsx',index = False)